In [1]:
!pip3 install ipython-cypher neo4j py2neo ipycytoscape >/dev/null

In [2]:
%env NEO4J_VERSION neo4j-community-4.1.1
%env NEO4J_PATH    neo4j-community-4.1.1

env: NEO4J_VERSION=neo4j-community-4.1.1
env: NEO4J_PATH=neo4j-community-4.1.1


In [3]:
!${NEO4J_PATH}/bin/neo4j status

Neo4j is not running


In [4]:
!curl http://localhost:7474/

{
  "bolt_routing" : "neo4j://localhost:7687",
  "transaction" : "http://localhost:7474/db/{databaseName}/tx",
  "bolt_direct" : "bolt://localhost:7687",
  "neo4j_version" : "4.1.1",
  "neo4j_edition" : "community"
}

In [5]:
# criando um adapter e verificando a conexão
from py2neo import Graph

graph = Graph("bolt://127.0.0.1:7687")

graph.delete_all()
print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.relationships)))


Number of nodes: 0 
Number of relationships: 0 


In [6]:
command = """
CALL apoc.load.json('dataset/json_with_relations.json') yield value
UNWIND value.OrderItems as items
MERGE (pedido:Pedido {id : value.order_id})
    ON CREATE SET pedido.nome = value.order_id,
                  pedido.status = value.order_status,
                  pedido.order_purchase_timestamp = value.order_purchase_timestamp,
                  pedido.order_approved_at = value.order_approved_at,
                  pedido.order_delivered_carrier_date = value.order_delivered_carrier_date,
                  pedido.order_delivered_customer_date = value.order_delivered_customer_date,
                  pedido.order_estimated_delivery_date = value.order_estimated_delivery_date
MERGE (cliente:Cliente {id : value.customer_id})
    ON CREATE SET cliente.nome = value.customer_id
MERGE (regiao : Regiao {id : value.Customer.customer_zip_code_prefix})
    ON CREATE SET regiao.nome = value.Customer.customer_zip_code_prefix
MERGE (cidade : Cidade {id : value.Customer.customer_city})
    ON CREATE SET cidade.nome = value.Customer.customer_city
MERGE (estado : Estado {id : value.Customer.customer_state})
    ON CREATE SET estado.nome = value.Customer.customer_state
MERGE (item_pedido: ItemPedido {id: value.order_id+'-'+items.order_item_id})
    ON CREATE SET item_pedido.nome = value.order_id+'-'+items.order_item_id,
                  item_pedido.preco = items.price,
                  item_pedido.shipping_limit_date = items.shipping_limit_date,
                  item_pedido.frete = items.freight_value
MERGE (produto: Produto {id : items.product_id})
    ON CREATE SET produto.nome = items.product_id,
                  produto.product_name_lenght = items.product.product_name_lenght,
                  produto.product_description_lenght = items.product.product_description_lenght,
                  produto.product_photos_qty = items.product.product_photos_qty,
                  produto.product_weight_g = items.product.product_weight_g,
                  produto.product_length_cm = items.product.product_length_cm,
                  produto.product_height_cm = items.product.product_height_cm,
                  produto.product_width_cm = items.product.product_width_cm
MERGE (fornecedor : Fornecedor {id : items.seller_id})
    ON CREATE SET fornecedor.nome = items.seller_id
MERGE (categoria : Categoria {id : items.product.product_category_name})
    ON CREATE SET categoria.nome = items.product.product_category_name
MERGE (regiaoFornecedor : Regiao {id : COALESCE(items.seller.seller_zip_code_prefix, "UNKNOWN")})
    ON CREATE SET regiaoFornecedor.nome = COALESCE(items.seller.seller_zip_code_prefix, "UNKNOWN")
MERGE (cidadeFornecedor : Cidade {id : COALESCE(items.seller.seller_city, "UNKNOWN")})
    ON CREATE SET cidadeFornecedor.nome = COALESCE(items.seller.seller_city, "UNKNOWN")
MERGE (cliente)-[:COMPROU {id_pedido : value.order_id}]->(produto)
MERGE (cliente)-[:MORA]->(regiao)
MERGE (regiao)-[:EH_PARTE_DE]->(cidade)
MERGE (cidade)-[:ESTA_LOCALIZADA]->(estado)
MERGE (produto)-[:EH_DE]->(item_pedido)
MERGE (produto)-[:EH_PRODUZIDO]->(fornecedor)
MERGE (produto)-[:EH_MONTADO]->(regiaoFornecedor)
MERGE (fornecedor)-[:PERTENCE]->(regiaoFornecedor)
MERGE (regiaoFornecedor)-[:EH_PARTE_DE]->(cidadeFornecedor)
MERGE (produto)-[:FOI_COMPRADO_POR]->(cliente)
MERGE (produto)-[:POSSUI]->(categoria)
MERGE (fornecedor)-[:FORNECE_CATEGORIA]->(categoria)
"""

result = graph.run(command)

In [7]:
# produtos mais vendidos de uma cidade
command = """
MATCH (cidade:Cidade)
WHERE cidade.id = "sao paulo"
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
WITH pro.id AS product, COUNT(cliAux) as qtdeProduto
RETURN product, qtdeProduto
ORDER BY qtdeProduto DESC
"""
result = graph.run(command)

print(result.to_table())


 product                          | qtdeProduto 
----------------------------------|-------------
 b1d207586fca400a2370d50a9ba1da98 |           9 
 aca2eb7d00ea1a7b8ebd4e68314663af |           7 
 6c3effec7c8ddba466d4f03f982c7aa3 |           7 
 0aabfb375647d9738ad0f7b4ea3653b1 |           6 
 595fac2a385ac33a80bd5114aec74eb8 |           5 
 b623b7cb05ee3248fbe4a6ecbeed79a4 |           5 
 5a848e4ab52fd5445cdc07aab1c40e48 |           4 
 e0cf79767c5b016251fe139915c59a26 |           4 
 19421075ae0b585f2dc13ff149e2119d |           3 
 803f77475e1b51b47f1bfec4f2ec353f |           3 
 e53e557d5a159f5aa2c5e995dfdf244b |           3 
 154e7e31ebfa092203795c972e5804a6 |           3 
 4914f8796af2ecd359fd8f44b9b92339 |           3 
 422879e10f46682990de24d770e7f83d |           3 
 4aa6014eceb682077f9dc4bffebc05b0 |           3 
 c72edc98e7d77f0c4c04ec80730c05f5 |           3 
 ec2d43cc59763ec91694573b31f1c29a |           3 
 7a10781637204d8d10485c71a6108a2e |           3 
 42a2c92a0979a949ca4

In [8]:
# categorias mais vendidas em uma cidade
command = """
MATCH (cidade:Cidade)
WHERE cidade.id = "sao paulo"
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proAux)-[:POSSUI]->(cat)
WHERE proAux.id = pro.id
WITH cat.id AS categoria, COUNT(pro) as qtdeProduto
RETURN categoria, qtdeProduto
ORDER BY qtdeProduto DESC
"""

result = graph.run(command)


print(result.to_table())


 categoria                                      | qtdeProduto 
------------------------------------------------|-------------
 cama_mesa_banho                                |          93 
 esporte_lazer                                  |          84 
 moveis_decoracao                               |          68 
 beleza_saude                                   |          61 
 brinquedos                                     |          60 
 relogios_presentes                             |          52 
 utilidades_domesticas                          |          51 
 informatica_acessorios                         |          39 
 automotivo                                     |          37 
 perfumaria                                     |          32 
 papelaria                                      |          30 
 cool_stuff                                     |          28 
 telefonia                                      |          27 
 ferramentas_jardim                             |      

In [9]:
# cidades que venderam mais produtos
command = """
MATCH (cidade:Cidade)
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[r:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
WITH cid.id as city, COUNT(pro.id) as qtdeProduto
RETURN city, qtdeProduto
ORDER BY qtdeProduto DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())


 city                  | qtdeProduto 
-----------------------|-------------
 sao paulo             |         871 
 rio de janeiro        |         418 
 belo horizonte        |         184 
 brasilia              |         133 
 salvador              |          95 
 campinas              |          86 
 curitiba              |          81 
 porto alegre          |          72 
 guarulhos             |          64 
 niteroi               |          54 
 sao bernardo do campo |          52 
 santo andre           |          50 
 sao jose dos campos   |          49 
 santos                |          47 
 fortaleza             |          43 
 recife                |          42 
 sao goncalo           |          40 
 goiania               |          39 
 osasco                |          38 
 sorocaba              |          38 
 ribeirao preto        |          35 
 belem                 |          34 
 jundiai               |          30 
 mogi das cruzes       |          29 
 uberlandia 

In [10]:
# Cidades que apresentaram o maior volume de vendas
command = """
MATCH (cidade:Cidade)
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proItem)-[:EH_DE]->(item)
WHERE proItem.id = pro.id
WITH cidade.id as city, SUM(item.preco) as valorVendas
RETURN city, ROUND(valorVendas*100)/100 as vendas
ORDER BY valorVendas DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())


 city                |    vendas 
---------------------|-----------
 sao paulo           |  418267.4 
 rio de janeiro      | 337955.72 
 belo horizonte      | 165375.42 
 brasilia            |  89549.75 
 salvador            |  79417.39 
 curitiba            |  59938.67 
 campinas            |  56264.15 
 santos              |  46434.36 
 porto alegre        |  43559.17 
 sao jose dos campos |  38976.93 
 fortaleza           |   36403.1 
 sao goncalo         |   36213.1 
 goiania             |  33612.63 
 santo andre         |  29766.74 
 guarulhos           |  28588.07 
 recife              |  25444.73 
 ribeirao preto      |  23271.97 
 niteroi             |  22057.05 
 uberlandia          |  22050.12 
 belem               |  20727.69 
 guaruja             |  20301.33 
 joao pessoa         |  20130.06 
 canoas              |  19994.31 
 sao vicente         |  18347.73 
 volta redonda       |  18254.32 



In [11]:
# Cidades que apresentaram o maior valor medio de frete cobrado em cada venda
command = """
MATCH (cidade:Cidade)
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proItem)-[:EH_DE]->(item)
WHERE proItem.id = pro.id
WITH cidade.id as city, AVG(item.frete) as valorFrete
RETURN city, ROUND(valorFrete*100)/100 as Frete
ORDER BY valorFrete DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())


 city                  |  Frete 
-----------------------|--------
 caputira              | 121.22 
 confresa              | 104.31 
 madre de deus         |  87.33 
 missal                |  84.71 
 martinopolis          |  83.04 
 santa rosa de viterbo |  79.42 
 capao do cipo         |  77.92 
 sobral                |  76.29 
 taciba                |  75.37 
 grajau                |  75.37 
 sao borja             |  74.79 
 potirendaba           |  68.13 
 itapoa                |  67.19 
 tucurui               |  64.17 
 vitoria da conquista  |   60.6 
 piracuruca            |  60.56 
 gilbues               |  60.51 
 sao luis do quitunde  |  59.77 
 novo hamburgo         |  59.35 
 cuite                 |  59.35 
 sao pedro dos ferros  |  58.14 
 casimiro de abreu     |  55.45 
 peixoto de azevedo    |  54.49 
 sao miguel do guama   |  54.11 
 itirucu               |  52.81 



In [12]:
# Cidades que apresentaram o maior valor medio de frete cobrado em cada venda
command = """
MATCH (cidade:Cidade)
WHERE cidade.id = "sao paulo"
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proAux)-[:POSSUI]->(cat)
WHERE proAux.id = pro.id
MATCH (proItem)-[:EH_DE]->(item)
WHERE proItem.id = proAux.id
WITH cidade.id as city, cat.id AS categoria, AVG(item.frete) as valorFrete
RETURN city, categoria, ROUND(valorFrete*100)/100 as Frete
ORDER BY valorFrete DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())

 city      | categoria                                      | Frete 
-----------|------------------------------------------------|-------
 sao paulo | moveis_cozinha_area_de_servico_jantar_e_jardim | 50.99 
 sao paulo | moveis_colchao_e_estofado                      | 40.98 
 sao paulo | malas_acessorios                               | 29.71 
 sao paulo | agro_industria_e_comercio                      | 26.99 
 sao paulo | eletrodomesticos_2                             | 24.15 
 sao paulo | moveis_escritorio                              | 23.85 
 sao paulo | sinalizacao_e_seguranca                        | 23.29 
 sao paulo | livros_interesse_geral                         | 22.38 
 sao paulo | cool_stuff                                     | 21.46 
 sao paulo | utilidades_domesticas                          | 21.19 
 sao paulo | ferramentas_jardim                             | 19.69 
 sao paulo | casa_construcao                                | 19.42 
 sao paulo | industria_comercio_e_